In [42]:
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn.preprocessing import StandardScaler
from sklearn import ensemble
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import xgboost
from xgboost import XGBRegressor

In [2]:
#경기
game_16 = pd.read_csv('2020빅콘테스트_스포츠투아이_제공데이터_경기_2016.csv', encoding="CP949")
game_17 = pd.read_csv('2020빅콘테스트_스포츠투아이_제공데이터_경기_2017.csv', encoding="CP949")
game_18 = pd.read_csv('2020빅콘테스트_스포츠투아이_제공데이터_경기_2018.csv', encoding="CP949")
game_19 = pd.read_csv('2020빅콘테스트_스포츠투아이_제공데이터_경기_2019.csv', encoding="CP949")
game_20 = pd.read_csv('2020빅콘테스트_스포츠투아이_제공데이터_경기_2020.csv', encoding="CP949")

# 팀타자
team_hitter_16 = pd.read_csv('2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2016.csv')
team_hitter_17 = pd.read_csv('2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2017.csv')
team_hitter_18 = pd.read_csv('2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2018.csv')
team_hitter_19 = pd.read_csv('2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2019.csv')
team_hitter_20 = pd.read_csv('2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2020.csv')

# 팀투수
team_pitcher_16 = pd.read_csv('2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2016.csv')
team_pitcher_17 = pd.read_csv('2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2017.csv')
team_pitcher_18 = pd.read_csv('2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2018.csv')
team_pitcher_19 = pd.read_csv('2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2019.csv')
team_pitcher_20 = pd.read_csv('2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2020.csv')

## preprocessing : train data

In [3]:
#선수 일정 별 몇 번째 경기인지 표시하는 함수
def game_from_date_to_game(data_set) :
    cnt = np.zeros((len(data_set),10), dtype='i8')
    
    team=["HH","HT","KT","LG","LT","NC","OB","SK","SS","WO"]
    for i in range(len(data_set)):
        col=0
        for j in team:
            if data_set.values[i,2]==j or data_set.values[i,3]==j:
                cnt[i,col]=np.count_nonzero(cnt[:i,col])+1
            col+=1

    return_data_set = pd.concat([data_set, pd.DataFrame(cnt.reshape(len(cnt),10),columns=team)],axis=1)
   
    return return_data_set.groupby('G_ID').sum().iloc[:,1:]

In [4]:
#일별로 팀의 경기수만 표시
date_game_16=game_from_date_to_game(game_16)
date_game_17=game_from_date_to_game(game_17)
date_game_18=game_from_date_to_game(game_18)
date_game_19=game_from_date_to_game(game_19)
date_game_20=game_from_date_to_game(game_20)

In [5]:
#경기별 개인 기록에 몇번째 경기인지 나타내는 변수를 추가하는 함수
def merge_date_game(date_game, data_set):
    HH_date_game=date_game.loc[date_game.HH>0,:].reset_index().loc[:,['G_ID','HH']]
    result=pd.merge(data_set.loc[data_set.T_ID=='HH',:], HH_date_game, on='G_ID')
    result.rename(columns={'HH': 'game'}, inplace=True)
    
    team=["HT","KT","LG","LT","NC","OB","SK","SS","WO"]
    for i in team:
        team_date_game=date_game.loc[date_game.loc[:,i]>0,:].reset_index().loc[:,['G_ID', i]]
        team_data=pd.merge(data_set.loc[data_set.T_ID==i,:], team_date_game, on='G_ID')
        team_data.rename(columns={i: 'game'}, inplace=True)
        result=pd.concat([result, team_data])
        
    return result

In [6]:
#위 함수로 변수 추가
hitter_game_16=merge_date_game(date_game_16, team_hitter_16)
hitter_game_17=merge_date_game(date_game_17, team_hitter_17)
hitter_game_18=merge_date_game(date_game_18, team_hitter_18)
hitter_game_19=merge_date_game(date_game_19, team_hitter_19)
hitter_game_20=merge_date_game(date_game_20, team_hitter_20)

pitcher_game_16=merge_date_game(date_game_16, team_pitcher_16)
pitcher_game_17=merge_date_game(date_game_17, team_pitcher_17)
pitcher_game_18=merge_date_game(date_game_18, team_pitcher_18)
pitcher_game_19=merge_date_game(date_game_19, team_pitcher_19)
pitcher_game_20=merge_date_game(date_game_20, team_pitcher_20)

In [7]:
# 팀 조합 90가지에 대한 시즌별 115경기 이후 안타율, 상대팀 안타율, 방어율, 상대팀 방어율 변수 생성

def transform_data(hitter_game, pitcher_game):
    win_p = pitcher_game.loc[pitcher_game.game>115,['T_ID','VS_T_ID', 'WLS']]
    game_count = win_p.groupby(['T_ID', 'VS_T_ID']).count().WLS
    for i in range(len(win_p)):
        if (win_p.iloc[i,2]=='W') : win_p.iloc[i,2]=1 
        else : win_p.iloc[i,2]=0
    win_p = win_p.groupby(['T_ID', 'VS_T_ID']).sum().reset_index()
    win_p['game_count'] = game_count.values
    win_p['wp'] = win_p.WLS / win_p.game_count
    
    avg = hitter_game_19.loc[hitter_game_19.game>115,['T_ID', 'HIT', 'AB']]
    avg = avg.groupby('T_ID').sum().reset_index()
    avg['AVG'] = avg.HIT / avg.AB
    
    era = pitcher_game.loc[pitcher_game.game>115,['T_ID', 'INN2', 'ER']]
    era = era.groupby('T_ID').sum().reset_index()
    era['ERA'] = (era.ER * 3 * 9) / era.INN2
    
    win_p = pd.merge(win_p.loc[:, ['T_ID', 'VS_T_ID', 'game_count', 'wp']], avg.loc[:, ['T_ID', 'AVG']], on='T_ID')
    win_p = pd.merge(win_p, era.loc[:, ['T_ID', 'ERA']], on='T_ID')
    win_p = pd.merge(win_p, avg.loc[:, ['T_ID', 'AVG']], left_on='VS_T_ID', right_on='T_ID')
    win_p = pd.merge(win_p, era.loc[:, ['T_ID', 'ERA']], left_on='VS_T_ID', right_on='T_ID')
    win_p = win_p.drop(['T_ID_y', 'T_ID'], axis=1)
    win_p.columns = ['T_ID', 'VS_T_ID', 'game_count', 'WP', 'AVG', 'ERA', 'VS_AVG', 'VS_ERA']
    
    return win_p

In [78]:
transform_data_16 = transform_data(hitter_game_16, pitcher_game_16)
transform_data_17 = transform_data(hitter_game_17, pitcher_game_17)
transform_data_18 = transform_data(hitter_game_18, pitcher_game_18)
transform_data_19 = transform_data(hitter_game_19, pitcher_game_19)

final=pd.concat([transform_data_16, transform_data_17, transform_data_18, transform_data_19])

In [80]:
# 각 시즌의 값을 나타내기 위한 연도 라벨링

def labeling_year(data, start_length, end_length, year):
    data.T_ID[start_length : end_length] = year + data.T_ID[start_length : end_length].values
    data.VS_T_ID[start_length : end_length] = year + data.VS_T_ID[start_length : end_length].values

labeling_year(final, 0, len(transform_data_16), '16')
labeling_year(final, len(transform_data_16), len(transform_data_16)+len(transform_data_17), '17')
labeling_year(final, len(transform_data_16)+len(transform_data_17), 
              len(transform_data_16)+len(transform_data_17)+len(transform_data_18), '18')
labeling_year(final, len(transform_data_16)+len(transform_data_17)+len(transform_data_18), 
              len(transform_data_16)+len(transform_data_17)+len(transform_data_18)+len(transform_data_19), '19')

<ipython-input-80-8878d44c7dcc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.T_ID[start_length : end_length] = year + data.T_ID[start_length : end_length].values
<ipython-input-80-8878d44c7dcc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.VS_T_ID[start_length : end_length] = year + data.VS_T_ID[start_length : end_length].values


In [27]:
train_x = final.iloc[:, 4:]
train_y = final.iloc[:, 3]

## preprocessing : test data

In [146]:
#아래부터 test data로 최종 예측 
left_game = pd.read_csv('test set.csv')
pred_avg = pd.read_csv('hitter_final_predict.csv')
pred_era = pd.read_csv('pitcher_final_predict.csv')

pred_avg_era = pd.merge(pred_avg, pred_era, on='T_ID')

In [147]:
#예측 avg, era를 남은 경기데이터에 merge하여 x 생성
test = left_game.loc[:,['T_ID', 'VS_T_ID','TB_SC']].groupby(['T_ID', 'VS_T_ID']).count().reset_index()

test = pd.merge(pd.merge(test, pred_avg, on='T_ID'), pred_era, on='T_ID')
test = pd.merge(pd.merge(test, pred_avg, left_on='VS_T_ID', right_on='T_ID'), pred_era, left_on='VS_T_ID', right_on='T_ID')
test = test.drop(['T_ID_y', 'T_ID'], axis=1)
test.columns = ['T_ID', 'VS_T_ID', 'game_count', 'AVG', 'ERA', 'VS_AVG', 'VS_ERA']

test_x = test.iloc[: , 3:]

## independent varaible scaling

In [31]:
scaler = StandardScaler()
train_x_scaled = scaler.fit_transform(train_x)
test_x_scaled = scaler.transform(test_x)

## choose best model [ridge, lasso, randomforest, xgboost]

In [129]:
param_ridge = {'alpha':np.arange(0.1,0.9,0.001)}
ridge = Ridge(normalize=True)
ridge_cv = RandomizedSearchCV(ridge, param_ridge, n_iter=20, n_jobs=-1, cv=5, random_state=79)
ridge_cv.fit(train_x_scaled, train_y)
ridge_cv.best_params_

{'alpha': 0.16000000000000006}

In [130]:
ridge_best = Ridge(alpha=ridge_cv.best_params_['alpha'], normalize=True)
ridge_best.fit(train_x_scaled, train_y)
pred_ridge = ridge_best.predict(train_x_scaled)
# mse_ridge
np.mean((pred_ridge-train_y)**2)

0.07693708280208307

In [131]:
param_lasso = {'alpha':np.arange(0.1,0.9,0.001)}
lasso = Lasso(normalize=True)
lasso_cv = RandomizedSearchCV(lasso, param_lasso, n_iter=20, n_jobs=-1, cv=5, random_state=79)
lasso_cv.fit(train_x_scaled, train_y)
lasso_cv.best_params_

{'alpha': 0.7450000000000006}

In [132]:
lasso_best = Lasso(alpha=lasso_cv.best_params_['alpha'], normalize=True)
lasso_best.fit(train_x_scaled, train_y)
pred_lasso = lasso_best.predict(train_x_scaled)
# mse_lasso
np.mean((pred_lasso-train_y)**2)

0.08317572278002783

In [133]:
param_grid = {'n_estimators':list(range(1,200)),
             'max_depth':list(range(1,100)),
             'max_features':['auto','sqrt','log2']}
randomized_cv = RandomizedSearchCV(RandomForestRegressor(), param_grid, n_iter=20, n_jobs=-1, cv=5, random_state=79)

rf_cv = randomized_cv.fit(train_x_scaled, train_y)
rf_cv.best_params_

{'n_estimators': 47, 'max_features': 'sqrt', 'max_depth': 5}

In [134]:
rf_best = RandomForestRegressor(n_estimators=rf_cv.best_params_['n_estimators'],
                               max_features=rf_cv.best_params_['max_features'],
                               max_depth=rf_cv.best_params_['max_depth'])
rf_best.fit(train_x_scaled, train_y)
pred_rf = rf_best.predict(train_x_scaled)
# mse_rf
np.mean((pred_rf-train_y)**2)

0.05699357639101262

In [135]:
param_xgb = {'learning_rate':np.arange(0.1,0.9,0.001),
             'max_depth':list(range(1,100)),
             'n_estimators':list(range(1,200))}
xgb = XGBRegressor(booster='gbtree', n_jobs=-1, early_stopping_rounds=20)
xgb_cv = RandomizedSearchCV(xgb, param_xgb, n_iter=20, n_jobs=-1, cv=5, random_state=79)
xgb_cv.fit(train_x_scaled, train_y)
xgb_cv.best_params_

[11:37:17] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




{'n_estimators': 1, 'max_depth': 25, 'learning_rate': 0.3290000000000002}

In [136]:
xgb_best = XGBRegressor(booster='gbtree', n_jobs=-1, early_stopping_rounds=20,
                        n_estimators=xgb_cv.best_params_['n_estimators'],
                        learning_rate=xgb_cv.best_params_['learning_rate'],
                        max_depth=xgb_cv.best_params_['max_depth'])
xgb_best.fit(train_x_scaled, train_y)
pred_xgb = xgb_best.predict(train_x_scaled)
# mse_xgb
np.mean((pred_xgb-train_y)**2)

[11:37:18] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




0.0529177872095082

## predict with best model : random forest regressor & xgboost

In [137]:
rf_best = RandomForestRegressor(n_estimators=rf_cv.best_params_['n_estimators'],
                               max_features=rf_cv.best_params_['max_features'],
                               max_depth=rf_cv.best_params_['max_depth'])
rf_best.fit(train_x_scaled, train_y)
pred_rf = rf_best.predict(test_x_scaled)

In [142]:
xgb_best = XGBRegressor(booster='gbtree', n_jobs=-1, early_stopping_rounds=20,
                        n_estimators=xgb_cv.best_params_['n_estimators'],
                        learning_rate=xgb_cv.best_params_['learning_rate'],
                        max_depth=xgb_cv.best_params_['max_depth'])
xgb_best.fit(train_x_scaled, train_y)
pred_xgb = xgb_best.predict(test_x_scaled)

[11:42:14] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [148]:
#predict
test['pred_rf'] = pred_rf
test['pred_xgb'] = pred_xgb

In [151]:
#가중평균 과정
test_pred = test.loc[:, ['T_ID', 'VS_T_ID', 'game_count', 'pred_rf','pred_xgb']]

test_pred['weighted_pred_rf'] = test_pred.game_count * test_pred.pred_rf
test_pred['weighted_pred_xgb'] = test_pred.game_count * test_pred.pred_xgb
team_pred = test_pred.groupby('T_ID').sum()
team_pred['pred_wp_rf'] = team_pred.weighted_pred_rf / team_pred.game_count
team_pred['pred_wp_wgb'] = team_pred.weighted_pred_xgb / team_pred.game_count

team_pred = team_pred.iloc[:,5:]

In [152]:
team_pred

,pred_wp_rf,pred_wp_wgb
T_ID,,
HH,0.379882,0.495114
HT,0.422385,0.497180
KT,0.504173,0.518345
LG,0.446562,0.509468
LT,0.452328,0.509172
NC,0.579986,0.539819
OB,0.570334,0.474821
SK,0.401247,0.460703
SS,0.475578,0.500070


In [154]:
# 올해의 경기 양상을 더 잘 반영하는 random forest regression으로 최종 predict

pred_final = team_pred.reset_index().iloc[:,:2]
pred_final

,T_ID,pred_wp_rf
0,HH,0.379882
1,HT,0.422385
2,KT,0.504173
3,LG,0.446562
4,LT,0.452328
5,NC,0.579986
6,OB,0.570334
7,SK,0.401247
8,SS,0.475578
9,WO,0.549309
